In [ ]:
from datasets import load_dataset
import cohere

In [ ]:
DATASET = "LeoZotos/immu_full"

In [ ]:
cohere_api_key = ""
hf_api_key = ""
with open("../tokens/COHERE.txt", "r") as f:
    cohere_api_key = f.read().strip()
with open("../tokens/HF_TOKEN.txt", "r") as f:
    hf_api_key = f.read().strip()
co = cohere.Client(cohere_api_key)

In [ ]:
data = load_dataset(DATASET, split="train", token=hf_api_key )

In [ ]:
def get_embeddings(batch):
    response = co.embed(
        texts=batch['Question_With_Options'],
        model='embed-multilingual-v3.0',
        input_type='search_query' # it is the query, not the document
    )
    return {"emb": response.embeddings}

data_with_embeddings = data.map(
    get_embeddings,
    batched=True,
    batch_size=96 
)

# Now `data_with_embeddings` is a new Dataset object with the 'emb' column
print(data_with_embeddings)

In [ ]:
print(data_with_embeddings[0]['emb'])

In [ ]:
data_with_embeddings.push_to_hub(
    repo_id=DATASET,
    commit_message="Add Cohere v3 embeddings to 'emb' column",
    token=hf_api_key,
    private=True
)